# 批量归一化 Batch Normalization

---

## 一、为什么需要批量归一化？

### 深层网络的问题

```
网络很深的时候（比如100层）

反向传播：梯度从上往下传
    顶层：梯度大 → 更新快 → 很快收敛
    底层：梯度小 → 更新慢 → 慢慢才动

问题：
    底层一动 → 底层提取的特征变了
    → 顶层之前学的东西全废了
    → 顶层又得重新学
    → 底层又动了
    → 顶层又得重新学
    → 恶性循环！收敛巨慢
```

**比喻：**

```
盖楼：
    地基（底层）还在慢慢调整
    上面的楼层（顶层）已经装修好了
    结果地基一动，上面全裂了，又得重新装修

批量归一化就是：
    让每一层的输出都保持在一个稳定的范围内
    这样不管底层怎么变，上面的变化都不会太剧烈
```

---

## 二、批量归一化到底做了什么？

### 核心操作

```
对每一层的输出：

第1步：算这个batch里的均值和方差
第2步：减均值，除方差（标准化成均值0、方差1）
第3步：再乘γ，加β（学一个新的均值和方差）

公式：
    输出 = γ × (输入 - 均值) / 方差 + β
    
    γ和β是可学习的参数
```

### 为什么要第3步？

```
第2步已经标准化了，为什么还要乘γ加β？

因为均值0方差1不一定是最好的分布
网络自己学出一个合适的均值和方差可能更好

γ学出来的就是"最佳方差"
β学出来的就是"最佳均值"

但因为γ和β变化很慢（受学习率控制）
所以整体分布是稳定的
```

**比喻：**

```
第2步 = 把所有人的身高统一成平均170cm
第3步 = 但篮球队可能需要平均185cm
        γ和β让网络自己决定最合适的"身高标准"
        
关键是：这个标准变化很缓慢，不会剧烈波动
```

---

## 三、放在哪个位置？

### 位置：卷积/全连接 → BN → 激活函数

```python
# 正确的顺序
nn.Conv2d(1, 6, kernel_size=5),
nn.BatchNorm2d(6),              # ← BN在卷积后面
nn.ReLU(),                       # ← 激活在BN后面

# 不要这样
nn.Conv2d(1, 6, kernel_size=5),
nn.ReLU(),                      
nn.BatchNorm2d(6),   # ✗ BN在后面错了！
```

**为什么在激活函数前面？**

```
ReLU会把负数变成0
如果先ReLU再BN：
    所有值都是正的 → 减均值后又变成有正有负 → 奇怪又回去了

先BN再ReLU：
    先调整好分布 → 再做非线性变换 → 合理
```

### 对卷积层 vs 全连接层

```
全连接层：对每个特征（每一列）做归一化
卷积层：对每个通道做归一化

但实际不需要管这个区别！
PyTorch会自动处理
你只需要知道用哪个：
    卷积层后面 → nn.BatchNorm2d
    全连接层后面 → nn.BatchNorm1d
```

---

## 四、训练 vs 推理的代码区别

# 先搞懂 net.train() 和 net.eval()

**训练 = 平时上课**

```
老师每节课都考一次小测验
每次小测验的平均分都不一样：
    第1次小测：班级平均75分
    第2次小测：班级平均82分
    第3次小测：班级平均68分
    ...

老师一边上课，一边偷偷记录：
    "这学期到目前为止，总体平均大概是76分"
    
这个"总体平均"每次小测后都更新一下
越来越准
```

**推理 = 期末考试**

```
期末考试只有1个同学来补考
就他1个人，你怎么算"班级平均分"？

算不了！1个人没法代表班级水平

怎么办？
    用这学期记录的"总体平均76分"就好了！
    这个数是靠平时很多次小测积累出来的
    已经很准了
```

---

## 翻译成神经网络

```
平时上课（训练）：
    每次来一个batch（比如256张图片）
    用这256张图片算均值和方差 ← 就像每次小测验的平均分
    同时更新"总体均值"和"总体方差" ← 就像记录学期总平均

期末考试（推理/预测）：
    可能只来1张图片
    1张图片算均值方差没意义
    直接用训练时积累的"总体均值"和"总体方差" ← 用学期总平均
```

---

## net.train() 和 net.eval() 在干什么？

```python
net.train()
```

```
告诉网络："现在是平时上课"
→ BN用当前这批学生（batch）的均值方差
→ 同时更新总体均值方差
```

```python
net.eval()
```

```
告诉网络："现在是期末考试"
→ BN用积累好的总体均值方差
→ 不再更新任何东西
```

---

## 为什么不能一直用batch的均值方差？

```
训练时：每次来256张图片，算均值方差 → 没问题，256张够多

推理时：可能就来1张图片
    1张图片的"均值" = 它自己
    1张图片的"方差" = 0
    
    你拿这个去做归一化？
    (x - x) / 0 = ???  → 直接炸了！
    
    所以推理时必须用一个"靠谱的"均值方差
    训练时积累的总体值就是那个"靠谱的"
```

---

## 你只需要记住的

```python
# 训练的时候，开头写这个
net.train()

# 要预测/评估的时候，写这个
net.eval()

# 就这两行！PyTorch自动处理所有细节
# 你不需要知道里面怎么算的
```

### 典型的训练代码

```python
for epoch in range(num_epochs):
    
    net.train()                      # ← 上课模式
    for X, y in train_loader:
        # 训练代码...
    
    net.eval()                       # ← 考试模式
    with torch.no_grad():
        for X, y in test_loader:
            # 评估代码...
```

---

## 一句话总结

```
net.train() = 告诉BN"用当前batch的统计量"
net.eval()  = 告诉BN"用训练时积累的统计量"

为什么？因为推理时可能只有1张图片，算不了均值方差
所以用训练时攒下来的

你只需要：训练前写train()，评估前写eval()
其他都不用管
```



---

## 五、代码实现

### BatchNorm的使用

```python
# 卷积层后面用 BatchNorm2d，参数 = 通道数
nn.BatchNorm2d(通道数)

# 全连接层后面用 BatchNorm1d，参数 = 特征数
nn.BatchNorm1d(特征数)
```

### LeNet + BatchNorm

```python
net = nn.Sequential(
    # === 第1组 ===
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.BatchNorm2d(6),              # BN！参数=输出通道数6
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    # === 第2组 ===
    nn.Conv2d(6, 16, kernel_size=5),
    nn.BatchNorm2d(16),             # BN！参数=输出通道数16
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    # === 全连接 ===
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120),
    nn.BatchNorm1d(120),            # BN！参数=输出特征数120
    nn.Sigmoid(),
    nn.Linear(120, 84),
    nn.BatchNorm1d(84),             # BN！参数=输出特征数84
    nn.Sigmoid(),
    nn.Linear(84, 10),              # 最后输出层不加BN
)
```

### 记忆规则

```
Conv2d(in, out, ...) → BatchNorm2d(out)
                                    ^^^
     参数就是前面Conv的输出通道数

Linear(in, out)      → BatchNorm1d(out)
                                    ^^^
     参数就是前面Linear的输出大小
```

---

## 六、BatchNorm的效果

```
不用BN：
    学习率只能用0.01这种小值
    训练慢，收敛需要很多epoch

用了BN：
    学习率可以用0.1甚至更大
    训练快，收敛更快
    
但最终精度差不多！
BN主要是加速训练，不是提升精度
```

---

## 七、BN和Dropout的关系

```
BN的一个副作用：有轻微的正则化效果
（因为每个batch的均值方差是随机的，相当于加了噪声）

所以：
    用了BN之后，Dropout的效果会减弱
    很多时候用了BN就不用Dropout了
    
实际中：
    ResNet等现代网络 → 用BN，不用Dropout
```

---

## 八、总结

### 需要记住的

```
1. BN做什么：减均值除方差，再乘γ加β
2. 放在哪里：卷积/全连接后面，激活函数前面
3. 效果：加速训练，允许更大学习率
4. 不改变最终精度，只是训练更快
```

### 代码模板

```python
# 卷积层后面
nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
nn.BatchNorm2d(out_ch),     # 参数 = 输出通道数
nn.ReLU(),

# 全连接层后面
nn.Linear(in_features, out_features),
nn.BatchNorm1d(out_features),  # 参数 = 输出特征数
nn.ReLU(),
```

### 训练时别忘了

```python
# 训练
net.train()     # BN用batch统计量

# 评估
net.eval()      # BN用全局统计量
```

### BatchNorm2d vs BatchNorm1d

| | BatchNorm2d | BatchNorm1d |
|:---|:---|:---|
| 用在哪后面 | Conv2d | Linear |
| 参数 | 输出通道数 | 输出特征数 |
| 数据维度 | 4维 | 2维 |

**一句话：Conv后面用2d，Linear后面用1d，参数都是前一层的输出大小。**